# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Puerto Ayora,97,EC,1601903086,89,-0.74,-90.35,66.00,5.01
1,Safi,0,MA,1601903087,60,32.30,-9.24,74.10,20.60
2,Port Alfred,0,ZA,1601903088,62,-33.59,26.89,73.56,6.13
3,Ruwi,20,OM,1601903089,66,23.60,58.55,89.60,6.93
4,Penzance,40,GB,1601903089,82,50.12,-5.54,55.40,27.51
...,...,...,...,...,...,...,...,...,...
569,Bambanglipuro,93,ID,1601903506,84,-7.95,110.28,76.82,12.19
570,Poronaysk,100,RU,1601903506,87,49.22,143.12,47.05,11.86
571,Acarí,30,PE,1601903507,54,-15.43,-74.62,69.98,5.10
572,Kargasok,100,RU,1601903508,95,59.06,80.86,32.38,8.12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configuring gmaps
gmaps.configure(api_key=g_key)

#Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

#Store humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plotting Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

#Creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

#Adding layer
fig.add_layer(heat_layer)

#Displaying figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the DataFrame to find your ideal weather condition:
#A max temperature lower than 80 degrees but higher than 70
#Wind speed less than 10 mph
#Zero cloudiness
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,Port Alfred,0,ZA,1601903088,62,-33.59,26.89,73.56,6.13
21,Tiznit Province,0,MA,1601903101,36,29.58,-9.50,79.11,7.76
23,Alaca,0,TR,1601903103,17,40.17,34.84,79.20,8.16
63,Chernenko,0,RU,1601903133,42,44.37,39.31,73.71,9.17
65,Castro,0,BR,1601902848,60,-24.79,-50.01,75.09,4.76
95,Cabo San Lucas,0,MX,1601903157,80,22.89,-109.91,73.99,1.01
131,Mount Isa,0,AU,1601903182,35,-20.73,139.50,73.40,9.71
264,Muzaffarabad,0,PK,1601903280,26,34.37,73.47,72.12,1.81
308,Kieta,0,PG,1601903310,77,-6.22,155.63,79.48,5.19
344,Kruisfontein,0,ZA,1601903332,65,-34.00,24.73,75.00,5.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store a variable named hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

#Result
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Port Alfred,ZA,-33.59,26.89,
21,Tiznit Province,MA,29.58,-9.50,
23,Alaca,TR,40.17,34.84,
63,Chernenko,RU,44.37,39.31,
65,Castro,BR,-24.79,-50.01,
95,Cabo San Lucas,MX,22.89,-109.91,
131,Mount Isa,AU,-20.73,139.50,
264,Muzaffarabad,PK,34.37,73.47,
308,Kieta,PG,-6.22,155.63,
344,Kruisfontein,ZA,-34.00,24.73,


In [8]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [20]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 2: Port Alfred.
Closest hotel in Port Alfred is Royal St Andrews Hotel.
------------
Retrieving Results for Index 21: Tiznit Province.
Missing field/result... skipping.
------------
Retrieving Results for Index 23: Alaca.
Missing field/result... skipping.
------------
Retrieving Results for Index 63: Chernenko.
Missing field/result... skipping.
------------
Retrieving Results for Index 65: Castro.
Closest hotel in Castro is Central Palace Hotel - Castro (PR).
------------
Retrieving Results for Index 95: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Waldorf Astoria Los Cabos Pedregal.
------------
Retrieving Results for Index 131: Mount Isa.
Closest hotel in Mount Isa is Copper City Motel.
------------
Retrieving Results for Index 264: Muzaffarabad.
Closest hotel in Muzaffarabad is Pearl Continental Muzaffarabad.
------------
Retrieving Results for Index 308: Kieta.
Missing field/result... skipping.
------------
Retrieving Results for Index 344: Kruisf

In [21]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Port Alfred,ZA,-33.59,26.89,Royal St Andrews Hotel
21,Tiznit Province,MA,29.58,-9.50,
23,Alaca,TR,40.17,34.84,
63,Chernenko,RU,44.37,39.31,
65,Castro,BR,-24.79,-50.01,Central Palace Hotel - Castro (PR)
95,Cabo San Lucas,MX,22.89,-109.91,Waldorf Astoria Los Cabos Pedregal
131,Mount Isa,AU,-20.73,139.50,Copper City Motel
264,Muzaffarabad,PK,34.37,73.47,Pearl Continental Muzaffarabad
308,Kieta,PG,-6.22,155.63,
344,Kruisfontein,ZA,-34.00,24.73,


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

#Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
2,Port Alfred,ZA,-33.59,26.89,Royal St Andrews Hotel
21,Tiznit Province,MA,29.58,-9.50,
23,Alaca,TR,40.17,34.84,
63,Chernenko,RU,44.37,39.31,
65,Castro,BR,-24.79,-50.01,Central Palace Hotel - Castro (PR)
95,Cabo San Lucas,MX,22.89,-109.91,Waldorf Astoria Los Cabos Pedregal
131,Mount Isa,AU,-20.73,139.50,Copper City Motel
264,Muzaffarabad,PK,34.37,73.47,Pearl Continental Muzaffarabad
308,Kieta,PG,-6.22,155.63,
344,Kruisfontein,ZA,-34.00,24.73,
